# Data Mining Project: Predicting Student Behavioral Disruptions


## Library Requirements
Install the required libraries specified in the `requirements.txt` file. You can do this using pip:
```bash
pip install -r requirements.txt
```


## Table of Contents
1. [Introduction](#introduction)
2. [Data Loading and Initial Exploration](#data-loading)
3. [Exploratory Data Analysis (EDA)](#eda)
4. [Hypothesis Testing](#hypothesis-testing)
5. [Feature Engineering](#feature-engineering)
6. [Model Development](#model-development)
7. [Model Evaluation and Interpretation](#model-evaluation)
8. [Result Cleanup](#result-cleanup)


## 1. Introduction
This notebook documents our team's effort to predict and analyze student behavioral disruptions to minimize in-class interruptions.

**Team Members:**  
**Customer:** Adam West

**Objectives:**
- Predict behavioral disruptions
- Identify anomalous patterns
- Provide clear interpretations


## 2. Data Loading and Initial Exploration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import f_oneway, ttest_ind, spearmanr, pearsonr
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import root_mean_squared_error  # Import the new function
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, r2_score, mean_squared_error
from sklearn.decomposition import PCA
from sklearn.metrics import root_mean_squared_error

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE







# Load datasets
bus_conduct = pd.read_csv('TTU Data - Bus Conduct.csv')
bus_conduct_updated = pd.read_csv('TTU Data Update - Bus Conduct.csv')
family_engagement = pd.read_csv('TTU Data - Family Engagement.csv')
disciplinary_referral = pd.read_csv('TTU Data - Disciplinary Referral.csv')
disciplinary_referral_updated = pd.read_csv('TTU Data Update - Disciplinary Referral.csv')
weather_df = pd.read_csv('weather.csv')

# Display the first few rows of each dataset for inspection
print("Bus Conduct Dataset:")
display(bus_conduct.head())
print("Bus Conduct Updated Dataset:")
display(bus_conduct_updated.head())

print("Family Engagement Dataset:")
display(family_engagement.head())

print("Disciplinary Referral Dataset:")
display(disciplinary_referral.head())

print("Disciplinary Referral Updated Dataset:")
display(disciplinary_referral_updated.head())


## 3. Exploratory Data Analysis (EDA)

In [ ]:

# Combine and clean datasets
disciplinary_referral_all = pd.concat([disciplinary_referral, disciplinary_referral_updated], ignore_index=True).drop_duplicates()
bus_conduct_all = pd.concat([bus_conduct, bus_conduct_updated], ignore_index=True).drop_duplicates()
# Convert date columns to datetime format
disciplinary_referral_all['Date of Incident'] = pd.to_datetime(disciplinary_referral_all['Date of Incident'], errors='coerce')
bus_conduct_all['Date of Incident'] = pd.to_datetime(bus_conduct_all['Date of Incident'], errors='coerce')
# Save combined datasets to CSV files
disciplinary_referral_all.to_csv("combined_disciplinary_referrals.csv", index=False)
bus_conduct_all.to_csv("combined_bus_conduct.csv", index=False)
# Check for missing values in the combined datasets
print("Missing values in Combined Bus Conduct Data:")
print(bus_conduct_all.isnull().sum())
print("\nMissing values in Combined Disciplinary Referral Data:")
print(disciplinary_referral_all.isnull().sum())


In [ ]:
# Add a 'Month' column for monthly analysis
disciplinary_referral_all['Month'] = disciplinary_referral_all['Date of Incident'].dt.month

# Group by month and count referrals
monthly_referrals = disciplinary_referral_all.groupby('Month').size()

# Plot monthly referral counts
plt.figure(figsize=(10, 5))
sns.barplot(x=monthly_referrals.index, y=monthly_referrals.values)
plt.title('Monthly Disciplinary Referrals (Updated Dataset)')
plt.xlabel('Month')
plt.ylabel('Number of Referrals')
plt.show()

In [ ]:

# Identify the top 10 students with the most referrals
frequent_students = disciplinary_referral_all['Student Identifier'].value_counts().head(10)

# Plot the top 10 students by referral count
plt.figure(figsize=(10, 5))
sns.barplot(y=frequent_students.index, x=frequent_students.values, orient='h')
plt.title('Top 10 Students by Number of Referrals (Updated Dataset)')
plt.xlabel('Number of Referrals')
plt.ylabel('Student Identifier')
plt.show()


In [ ]:
# Function to categorize time of day
def categorize_time(time_str):
    if pd.isna(time_str): return "Unknown"
    time_str = time_str.lower().strip()
    if "before school" in time_str: return "Before School"
    elif any(t in time_str for t in ["8:00am", "9:00am", "10:00am", "11:00am"]): return "Morning"
    elif any(t in time_str for t in ["12:00pm", "1:00pm"]): return "Early Afternoon"
    elif any(t in time_str for t in ["2:00pm", "3:00pm"]): return "Late Afternoon"
    elif "after school" in time_str: return "After School"
    else: return "Other"

# Define the order of time categories
time_order = ["Before School", "Morning", "Early Afternoon", "Late Afternoon", "After School", "Other"]

# Apply time categorization to the dataset
disciplinary_referral_all["Time_Category"] = disciplinary_referral_all["Time of the Day the behavior occurred?"].apply(categorize_time)

# Set ordered categories for plotting
disciplinary_referral_all["Time_Category"] = pd.Categorical(
    disciplinary_referral_all["Time_Category"],
    categories=time_order,
    ordered=True
)

# Group data by grade level and time category
grouped = disciplinary_referral_all.groupby(["Grade_Level", "Time_Category"]).size().unstack().fillna(0)

# Sort grades for display
grouped = grouped.sort_index()

# Plot a heatmap of referrals by grade level and time of day
plt.figure(figsize=(12, 6))
sns.heatmap(grouped, annot=True, fmt='d', cmap="YlGnBu")
plt.title("Disciplinary Referrals by Grade Level and Time of Day (Cleaned)")
plt.xlabel("Time of Day")
plt.ylabel("Grade Level")
plt.tight_layout()
plt.show()

In [ ]:
# Convert weather data datetime column to datetime format
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'], errors='coerce')

# Group referrals by date and count
referrals_per_day = disciplinary_referral_all.groupby('Date of Incident').size().reset_index(name='referral_count')

# Merge referral counts with weather data
merged_df = pd.merge(referrals_per_day, weather_df, how='inner', left_on='Date of Incident', right_on='datetime')
merged_df.drop(columns=['datetime'], inplace=True)

# Categorize temperature into bins
bins = [0, 50, 70, 100]
labels = ['Cold (<50°F)', 'Mild (50–70°F)', 'Hot (>70°F)']
merged_df['Temp_Category'] = pd.cut(merged_df['temp'], bins=bins, labels=labels)

# Calculate average referrals by temperature category
binned_referrals = merged_df.groupby('Temp_Category')['referral_count'].mean().reset_index()

# Plot average referrals by temperature category
plt.figure(figsize=(8, 5))
sns.barplot(data=binned_referrals, x='Temp_Category', y='referral_count', palette='coolwarm')
plt.title('Average Referrals by Temperature (Updated)')
plt.ylabel('Average Number of Referrals')
plt.xlabel('Temperature Range')
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

## 3.5 Expanded EDA: Additional Insights and Correlations

## Expanded EDA: Additional Insights and Correlations

### Referrals by School or Staff

In [ ]:
# Analyze referrals by school or staff
staff_referral_counts = disciplinary_referral_all['Please select your school'].value_counts().head(10)

# Plot the top 10 schools by referral count
plt.figure(figsize=(10, 5))
sns.barplot(y=staff_referral_counts.index, x=staff_referral_counts.values, orient='h')
plt.title("Top 10 Schools by Referral Count")
plt.xlabel("Referrals Issued")
plt.ylabel("School")
plt.tight_layout()
plt.show()

Weather and Referral Type (e.g Fighting)

In [ ]:
# Analyze weather and referral type (e.g., fighting-related referrals)
fighting_referrals = disciplinary_referral_all[
    disciplinary_referral_all['Select the Major Referral'].str.contains('fight', na=False, case=False)
]

# Group fighting-related referrals by date
fight_days = fighting_referrals.groupby('Date of Incident').size().reset_index(name='fight_referrals')

# Merge fighting-related referrals with weather data
weather_fights = pd.merge(fight_days, weather_df, left_on='Date of Incident', right_on='datetime', how='inner')

# Plot daily fight referrals vs. temperature
plt.figure(figsize=(10, 5))
sns.scatterplot(data=weather_fights, x='temp', y='fight_referrals')
plt.title("Daily Fight Referrals vs. Temperature")
plt.xlabel("Temperature")
plt.ylabel("Number of Fight-Related Referrals")
plt.grid(True)
plt.tight_layout()
plt.show()

### Bus Conduct vs. Classroom Referrals

In [ ]:
# Analyze the relationship between bus conduct incidents and classroom referrals
bus_counts = bus_conduct_all['Student Identifier'].value_counts().reset_index()
bus_counts.columns = ['Student Identifier', 'Bus_Incidents']

referral_counts = disciplinary_referral_all['Student Identifier'].value_counts().reset_index()
referral_counts.columns = ['Student Identifier', 'Referrals']

# Merge bus conduct and referral data
merged_behavior = pd.merge(bus_counts, referral_counts, on='Student Identifier', how='outer').fillna(0)

# Plot referrals vs. bus conduct incidents
plt.figure(figsize=(8, 6))
sns.scatterplot(data=merged_behavior, x='Bus_Incidents', y='Referrals')
plt.title('Referrals vs. Bus Conduct Incidents')
plt.xlabel('Bus Conduct Incidents')
plt.ylabel('Referrals')
plt.grid(True)
plt.tight_layout()
plt.show()

### Weekday Trends in Referrals

In [ ]:
# Analyze weekday trends in referrals
disciplinary_referral_all['Weekday'] = disciplinary_referral_all['Date of Incident'].dt.day_name()

# Define the order of weekdays for plotting
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

# Count referrals by weekday
weekday_counts = disciplinary_referral_all['Weekday'].value_counts().reindex(weekday_order)

# Plot referrals by day of the week
plt.figure(figsize=(8, 4))
sns.barplot(x=weekday_counts.index, y=weekday_counts.values)
plt.title('Referrals by Day of the Week')
plt.ylabel('Number of Referrals')
plt.xlabel('Day')
plt.tight_layout()
plt.show()

## 4. Hypothesis Testing
This section explores several data-driven hypotheses relevant to predicting and minimizing in-class behavioral disruptions. The following tests were conducted using statistical methods such as t-tests, ANOVA, and correlation analyses.

--- 

### H1: Referral Frequency Increases Near Testing Season

**Hypothesis:** March and April have significantly higher referral counts due to testing-related stress.

**Test Type:** One-way ANOVA  
**Rationale:** Compare monthly referral averages across months.

In [ ]:
# Drop NaNs and extract month
monthly_ref = disciplinary_referral_all.dropna(subset=['Date of Incident']).copy()
monthly_ref['Month'] = monthly_ref['Date of Incident'].dt.month

# Calculate number of referrals per student per month
monthly_student_referrals = (
    monthly_ref.groupby(['Month', 'Student Identifier'])
    .size()
    .reset_index(name='Referral Count')
)

# Create a list of referral counts per month for ANOVA, filter groups with more than 1 value and variance > 0
monthly_groups = [
    group['Referral Count'].values
    for _, group in monthly_student_referrals.groupby('Month')
    if len(group) > 1 and group['Referral Count'].var() > 0
]

# Perform ANOVA if valid groups exist
if len(monthly_groups) >= 2:
    f_stat, p_value = f_oneway(*monthly_groups)
    print(f"F-statistic: {f_stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    if not pd.isna(f_stat) and p_value < 0.05:
        print("Significant differences exist between monthly referral counts.")
    elif not pd.isna(f_stat):
        print("No significant difference between months.")
    else:
        print("ANOVA returned NaN. Check your data again for consistency.")
else:
    print("Not enough valid monthly groups for ANOVA.")



---

### H2: Bus Misconduct is Associated with More In-Class Referrals

**Hypothesis:** Students with bus conduct incidents have significantly more in-class referrals than those without.

**Test Type:** Welch’s t-test (independent two-sample t-test)  
**Rationale:** Compare mean referral counts between two groups (bus incident vs. no bus incident).

In [ ]:
# Total referrals per student
referral_counts = disciplinary_referral_all['Student Identifier'].value_counts().reset_index()
referral_counts.columns = ['Student Identifier', 'Total_Referrals']

# Total bus incidents per student
bus_counts = bus_conduct_all['Student Identifier'].value_counts().reset_index()
bus_counts.columns = ['Student Identifier', 'Bus_Incidents']

# Merge datasets
behavior_merge = pd.merge(referral_counts, bus_counts, on='Student Identifier', how='outer').fillna(0)

# Create two groups
bus_yes = behavior_merge[behavior_merge['Bus_Incidents'] > 0]['Total_Referrals']
bus_no = behavior_merge[behavior_merge['Bus_Incidents'] == 0]['Total_Referrals']

# Run t-test
t_stat, p_value = ttest_ind(bus_yes, bus_no, equal_var=False)

print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")
if p_value < 0.05:
    print("Statistically significant difference found.")
else:
    print("No significant difference.")

---

### H3: Family Engagement Negatively Correlates with Referrals

**Hypothesis:** Higher family engagement is associated with fewer referrals.

**Test Type:** Spearman Correlation  
**Rationale:** Non-parametric test of ordinal survey response counts vs. referral totals.

In [ ]:
# Clean columns and estimate total referrals per school
referrals_by_school = disciplinary_referral_all['Please select your school'].value_counts().reset_index()
referrals_by_school.columns = ['School', 'Total_Referrals']

# Survey counts per school
engagement_by_school = family_engagement['Please check which school your child/children attends.'].value_counts().reset_index()
engagement_by_school.columns = ['School', 'Engagement_Survey_Responses']

school_merge = pd.merge(referrals_by_school, engagement_by_school, on='School', how='inner')

# Spearman correlation
corr, p_value = spearmanr(school_merge['Engagement_Survey_Responses'], school_merge['Total_Referrals'])
print(f"Spearman correlation: {corr:.4f}\nP-value: {p_value:.4f}")
if p_value < 0.05:
    print("Significant negative/positive correlation.")
else:
    print("No significant correlation found.")


---

### H4: Referral Count Differs by Grade Level

**Hypothesis:** Certain grades have significantly more referrals.

**Test Type:** One-way ANOVA  
**Rationale:** Compare referral frequency across grades.


In [ ]:
# Ensure Grade_Level is numeric and drop rows with missing Grade_Level
disciplinary_referral_all['Grade_Level'] = pd.to_numeric(disciplinary_referral_all['Grade_Level'], errors='coerce')
ref_by_grade = disciplinary_referral_all.dropna(subset=['Grade_Level'])

# Calculate number of referrals per student per grade
grade_student_referrals = (
    ref_by_grade.groupby(['Grade_Level', 'Student Identifier'])
    .size()
    .reset_index(name='Referral Count')
)

# Create a list of referral counts per grade for ANOVA, filtering for groups with variance > 0
grade_groups = [
    group['Referral Count'].values
    for _, group in grade_student_referrals.groupby('Grade_Level')
    if len(group) > 1 and group['Referral Count'].var() > 0
]

# Perform ANOVA if there are at least two valid grade groups
if len(grade_groups) >= 2:
    f_stat, p_value = f_oneway(*grade_groups)
    print(f"F-statistic: {f_stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    if not pd.isna(f_stat) and p_value < 0.05:
        print("Referral rates differ significantly across grades.")
    elif not pd.isna(f_stat):
        print("No significant difference in referrals between grades.")
    else:
        print("ANOVA returned NaN. Check your data for consistency.")
else:
    print("Not enough valid grade groups for ANOVA.")



---

### H5: Referral Volume Correlates with Weather Factors

**Hypothesis:** Temperature and humidity levels are associated with referral counts.

**Test Type:** Pearson Correlation  
**Rationale:** Compare numeric weather features against referral count per day.


In [ ]:
# Prepare merged dataset
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'], errors='coerce')
daily_ref = disciplinary_referral_all.groupby('Date of Incident').size().reset_index(name='referral_count')
daily_weather = pd.merge(daily_ref, weather_df, left_on='Date of Incident', right_on='datetime', how='inner')

# Pearson correlations
for var in ['temp', 'humidity']:
    corr, p = pearsonr(daily_weather['referral_count'], daily_weather[var])
    print(f"Pearson correlation between referrals and {var}: {corr:.4f} (p = {p:.4f})")

## Hypothesis Testing Summary and Interpretation

This section summarizes the results of five statistical tests aimed at identifying significant patterns and relationships related to student behavioral referrals. These insights support the ultimate goal of targeting interventions and informing stakeholder decision-making.

---

### **H1: Referral Frequency Increases Near Testing Season**
- **F-statistic:** `NaN`  
- **P-value:** `NaN`  
- **Result:** *Test could not be performed due to insufficient or non-variable data.*

**Interpretation:**  
Referral data for summer months (June, July) was missing, and some months had uniform or single-entry referral counts. This lack of statistical variance violated ANOVA assumptions, making the test invalid.

**Implication:**  
A follow-up study with more balanced month-by-month data could explore whether academic testing stress correlates with behavioral disruptions.

---

### **H2: Bus Misconduct is Associated with More In-Class Referrals**
- **T-statistic:** `-4.9906`  
- **P-value:** `0.0000`  
- **Result:** *Statistically significant difference found.*

**Interpretation:**  
Students with at least one bus conduct incident had significantly higher referral counts compared to those without. The large negative t-statistic confirms that the two group means are meaningfully different.

**Implication:**  
Bus misconduct is a strong predictor of general behavioral issues. Students flagged for bus incidents may benefit from preemptive behavioral interventions or monitoring in the classroom.

---

### **H3: Family Engagement Negatively Correlates with Referrals**
- **Spearman correlation:** `-0.8000`  
- **P-value:** `0.2000`  
- **Result:** *Not statistically significant.*

**Interpretation:**  
While a strong negative correlation was observed (suggesting that higher engagement corresponds with fewer referrals), the p-value was not below the 0.05 threshold for significance. This may be due to small sample size (school-level aggregation).

**Implication:**  
There is potential evidence that increased family involvement could reduce behavior issues. Larger or student-level datasets may yield stronger conclusions.

---

### **H4: Referral Count Differs by Grade Level**
- **F-statistic:** `NaN`  
- **P-value:** `NaN`  
- **Result:** *Test could not be performed due to insufficient group variability.*

**Interpretation:**  
Some grades had extremely low referral counts or lacked variation, violating ANOVA requirements. As with H1, this results in an invalid test.

**Implication:**  
FIX THIS LATER

---

### **H5: Weather Correlation with Referral Volume**
- **Temperature Correlation (Pearson):** `0.1636` (p = `0.0455`)  
- **Humidity Correlation (Pearson):** `0.1566` (p = `0.0557`) 

**Interpretation:**  
There is a **weak but statistically significant** positive correlation between **temperature** and **referral volume**, indicating referrals tend to rise on warmer days. The correlation with humidity is borderline and not statistically significant at α = 0.05.

**Implication:**  
Environmental stressors such as heat may contribute to behavioral issues. Schools could explore increased support or schedule adjustments during high-heat periods.

---

### Overall Recommendations:

- **Focus future models on bus conduct as a predictive feature.**
- **Consider temperature as a situational risk factor.**
- **Explore family engagement strategies to reduce referrals.**


## Initial Recommendations for School Stakeholders

Based on data-driven hypothesis testing and exploratory analysis, the following recommendations are proposed to help school administrators reduce behavioral disruptions and improve classroom environments:

---

### 1. Monitor Students with Bus Conduct Incidents
> **Why:** Students with bus conduct violations had *statistically significantly higher* classroom referral rates.

**Recommendations:**
- Flag students with bus write-ups for early behavioral intervention or counseling.
- Integrate bus conduct records into early warning systems.
- Enhance collaboration between bus drivers and classroom teachers.

---

### 2. Plan for Heat-Related Behavior Increases
> **Why:** Referral counts showed a *significant positive correlation* with higher temperatures.

**Recommendations:**
- Improve classroom cooling access and hydration breaks during hot weather.
- Train teachers in managing heat-induced student irritability.
- Monitor referrals during heatwaves and adjust scheduling if needed.

---

### 3. Use Family Engagement as a Soft Predictor
> **Why:** A strong negative (though not statistically significant) correlation was observed between family engagement and referrals.

**Recommendations:**
- Encourage increased parental participation in school events and surveys.
- Use engagement metrics to target school-specific outreach strategies.
- Offer incentives for family involvement in education and discipline policies.

---

### 4. Improve Data Collection for Temporal Analysis
> **Why:** Missing data for summer months and inconsistent grade-level information made trend testing inconclusive.

**Recommendations:**
- Ensure consistent referral logging across all months, including summer sessions.
- Standardize grade-level encoding across datasets.
- Capture time-of-day and seasonal data with higher granularity.

---

### 5. Prioritize Multivariate Risk Profiling
> **Why:** Bus behavior, weather, and family engagement all show predictive potential.

**Recommendations:**
- Develop internal dashboards combining behavior, attendance, engagement, and weather.
- Group students by intervention tiers using these combined indicators.
- Explore predictive modeling to proactively identify at-risk students.

---

These recommendations are intended to guide practical changes and inform predictive modeling efforts that follow in subsequent sections.


## 5. Feature Engineering
In this section we will create a student-week model dataset that aggregates student behavior data on a weekly basis. This will help us analyze trends and patterns in student behavior over time.


In [ ]:
#  Merging all the datasets into one model ready dataset
# Reload datasets
original_ref = pd.read_csv("TTU Data - Disciplinary Referral.csv")
update_ref = pd.read_csv("TTU Data Update - Disciplinary Referral.csv")
original_bus = pd.read_csv("TTU Data - Bus Conduct.csv")
update_bus = pd.read_csv("TTU Data Update - Bus Conduct.csv")
family_engagement = pd.read_csv("TTU Data - Family Engagement.csv")
weather = pd.read_csv("weather.csv")

# Merge referrals
all_ref_columns = list(set(original_ref.columns).union(set(update_ref.columns)))
original_ref = original_ref.reindex(columns=all_ref_columns)
update_ref = update_ref.reindex(columns=all_ref_columns)
full_ref = pd.concat([original_ref, update_ref], ignore_index=True).drop_duplicates()

# Merge bus conduct
all_bus_columns = list(set(original_bus.columns).union(set(update_bus.columns)))
original_bus = original_bus.reindex(columns=all_bus_columns)
update_bus = update_bus.reindex(columns=all_bus_columns)
full_bus = pd.concat([original_bus, update_bus], ignore_index=True).drop_duplicates()

# Preprocess dates
full_ref['Date of Incident'] = pd.to_datetime(full_ref['Date of Incident'], errors='coerce')
full_bus['Date of Incident'] = pd.to_datetime(full_bus['Date of Incident'], errors='coerce')
weather['datetime'] = pd.to_datetime(weather['datetime'], errors='coerce')

# STEP 1: Create 'Week' columns
full_ref['Week'] = full_ref['Date of Incident'].dt.to_period('W').apply(lambda r: r.start_time)
full_bus['Week'] = full_bus['Date of Incident'].dt.to_period('W').apply(lambda r: r.start_time if not pd.isnull(r) else None)
# STEP 2: Aggregate referral and bus incidents per student per week
ref_agg = full_ref.groupby(['Student Identifier', 'Week']).size().reset_index(name='weekly_referrals')
bus_agg = full_bus.groupby(['Student Identifier', 'Week']).size().reset_index(name='weekly_bus_incidents')

# STEP 3: Extract basic student metadata
student_meta = full_ref.drop_duplicates('Student Identifier')[['Student Identifier', 'Grade_Level', 'Gender', 'Ethnicity', 'LunchStatus']]

# STEP 4: Normalize and prepare engagement survey data
engagement_data = family_engagement.rename(columns=lambda x: x.strip())
if 'Student Identifier' not in engagement_data.columns:
    for col in engagement_data.columns:
        if 'student' in col.lower() and 'id' in col.lower():
            engagement_data.rename(columns={col: 'Student Identifier'}, inplace=True)
            break

# STEP 5: Merge referral and bus data
student_weeks = pd.merge(ref_agg, bus_agg, on=['Student Identifier', 'Week'], how='outer').fillna(0)

# STEP 6: Add student demographic data
student_weeks = pd.merge(student_weeks, student_meta, on='Student Identifier', how='left')

# STEP 7: Add engagement survey data
student_weeks = pd.merge(student_weeks, engagement_data, on='Student Identifier', how='left')

# STEP 8: Add weekly weather aggregates
weather['Week'] = weather['datetime'].dt.to_period('W').apply(lambda r: r.start_time)
weather_weekly = weather.groupby('Week').agg({
    'temp': 'mean',
    'humidity': 'mean',
    'precip': 'mean',
    'sealevelpressure': 'mean',
    'windgust': 'mean'
}).reset_index()
student_weeks = pd.merge(student_weeks, weather_weekly, on='Week', how='left')

# STEP 9: Create target variable
student_weeks = student_weeks.sort_values(by=['Student Identifier', 'Week'])
student_weeks['referral_next_week'] = student_weeks.groupby('Student Identifier')['weekly_referrals'].shift(-1)
student_weeks['referral_next_week'] = (student_weeks['referral_next_week'] > 0).astype(int)

# Display the result
print("Model-Ready Dataset Preview:")
print(student_weeks.head())
print("\nShape:", student_weeks.shape)
print("Columns:", student_weeks.columns.tolist())

student_weeks.to_csv("model_ready_student_weeks.csv", index=False)


## 6. Model Development

In this section, we implement logistic regression, linear regression, and an advanced Random Forest model.

### Loading the data and training/testing split

In [ ]:
student_weeks = pd.read_csv("model_ready_student_weeks.csv")
student_weeks.dropna(subset=['referral_next_week'], inplace=True)

features = ['weekly_referrals', 'weekly_bus_incidents', 'Grade_Level', 'Gender',
            'Ethnicity', 'LunchStatus', 'temp', 'humidity', 'precip', 'sealevelpressure', 'windgust']

X = student_weeks[features]
y_classification = student_weeks['referral_next_week']
y_regression = student_weeks['weekly_referrals']

X_train, X_test, y_clf_train, y_clf_test, y_reg_train, y_reg_test = train_test_split(
    X, y_classification, y_regression, test_size=0.2, random_state=42)

### Preprocessing Pipeline

In [ ]:
# Preprocessing pipelines
numeric_features = ['weekly_referrals', 'weekly_bus_incidents', 'Grade_Level', 'temp', 'humidity', 'precip', 'sealevelpressure', 'windgust']
categorical_features = ['Gender', 'Ethnicity', 'LunchStatus']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])


### Model Pipeline

In [ ]:
# Model pipelines
pipelines = {
    'LogisticRegression': ImbPipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('clf', LogisticRegression(max_iter=1000, random_state=42))
    ]),
    'LinearRegression': Pipeline([
        ('preprocessor', preprocessor),
        ('clf', LinearRegression())
    ]),
    'NeuralNetwork': ImbPipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('clf', MLPClassifier(hidden_layer_sizes=(50,), max_iter=300, random_state=42))
    ])
}

### Parameter Grids

In [ ]:
# Parameter grids
param_grids = {
    'LogisticRegression': {
        'clf__C': [0.01, 0.1, 1.0, 10.0],
        'clf__penalty': ['l2'],
        'clf__solver': ['lbfgs']
    },
    'LinearRegression': {
        'clf__fit_intercept': [True, False],
        'clf__positive': [False, True]
    },
    'NeuralNetwork': {
        'clf__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (64, 64, 32)],
        'clf__activation': ['relu', 'tanh'],
        'clf__solver': ['adam'],
        'clf__alpha': [0.0001, 0.001],
        'clf__learning_rate': ['constant', 'adaptive'],
        'clf__early_stopping': [True],
        'clf__n_iter_no_change': [5]
    }
}

# GridSearchCV runner
def run_grid_searches(X_train, y_train, pipelines):
    best_models = {}
    for model_name, pipeline in pipelines.items():
        if model_name not in param_grids:
            print(f"Skipping model: {model_name}")
            continue

        print(f"\nTuning hyperparameters for: {model_name}...")

        grid = GridSearchCV(
            pipeline,
            param_grid=param_grids[model_name],
            cv=5,
            scoring='f1_macro' if model_name != 'LinearRegression' else 'r2',
            n_jobs=-1,
            verbose=1
        )

        grid.fit(X_train, y_train)
        best_models[model_name] = grid

        print(f"\nBest Params for {model_name}: {grid.best_params_}")
        print(f"Best Score: {grid.best_score_:.4f}")

    return best_models

### DBSCAN Clustering Implementation

In [ ]:
def evaluate_dbscan(X_raw):
    # Preprocess the data
    X_numeric = X_raw.select_dtypes(include=[np.number])
    X_numeric = pd.DataFrame(SimpleImputer(strategy="mean").fit_transform(X_numeric), columns=X_numeric.columns)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_numeric)

    # Parameter ranges
    eps_vals = np.arange(0.1, 1.1, 0.1)
    min_samples_vals = [3, 5, 7]

    best_score = -1
    best_model = None
    best_params = None

    # Iterate over parameter combinations
    for eps in eps_vals:
        for min_samples in min_samples_vals:
            model = DBSCAN(eps=eps, min_samples=min_samples)
            labels = model.fit_predict(X_scaled)
            n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

            if n_clusters < 2:
                print(f"Skipped eps={eps:.1f}, min_samples={min_samples} — only {n_clusters} cluster(s) detected")
                continue

            score = silhouette_score(X_scaled, labels)
            print(f"Checked eps={eps:.1f}, min_samples={min_samples} → Silhouette Score: {score:.4f}")

            if score > best_score:
                best_score = score
                best_model = model
                best_params = {'eps': eps, 'min_samples': min_samples}

    if best_model:
        print("\nBest DBSCAN Params:", best_params)
        print("Best Silhouette Score:", best_score)
    else:
        print("No valid DBSCAN clustering found. Adjust parameter ranges or examine dataset scale.")

    return best_model, best_score

## 7. Model Evaluation and Interpretation


In [ ]:
def evaluate_model_results(best_models, X_test, y_clf_test, y_reg_test):
    for name, model in best_models.items():
        print(f"\nEvaluation Results for {name}")

        y_pred = model.predict(X_test)

        # Always compute RMSE and R² for consistency
        try:
            rmse = root_mean_squared_error(y_reg_test, y_pred)
            r2 = r2_score(y_reg_test, y_pred)
            print(f"RMSE: {rmse:.4f}")
            print(f"R² Score: {r2:.4f}")
        except Exception as e:
            print("[RMSE/R² Calculation Error]:", e)

        if name != 'LinearRegression':
            try:
                cm = confusion_matrix(y_clf_test, y_pred)
                prec = precision_score(y_clf_test, y_pred)
                rec = recall_score(y_clf_test, y_pred)
                f1 = f1_score(y_clf_test, y_pred)

                print(f"Precision: {prec:.4f}")
                print(f"Recall: {rec:.4f}")
                print(f"F1 Score: {f1:.4f}")

                plt.figure(figsize=(5, 4))
                sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
                plt.title(f"{name} — Confusion Matrix")
                plt.xlabel("Predicted")
                plt.ylabel("Actual")
                plt.tight_layout()
                plt.show()
            except Exception as e:
                print("[Classification Metrics Error]:", e)
        else:
            # Regression scatter plot only for linear regression
            plt.figure(figsize=(6, 4))
            plt.scatter(y_reg_test, y_pred, alpha=0.3)
            plt.title(f'{name} — Actual vs. Predicted Referrals')
            plt.xlabel("Actual Referrals")
            plt.ylabel("Predicted Referrals")
            plt.grid(True)
            plt.tight_layout()
            plt.show()

# Usage:
best_models = run_grid_searches(X_train, y_clf_train, pipelines)
evaluate_model_results(best_models, X_test, y_clf_test, y_reg_test)

### DBSCAN Clustering Results

In [ ]:
def visualize_dbscan_clusters(dbscan_model, X_raw, preprocessor):
    if dbscan_model is None:
        print("DBSCAN model is None — check if a valid model was returned from evaluate_dbscan().")
        return

    X_preprocessed = preprocessor.fit_transform(X_raw)
    labels = dbscan_model.fit_predict(X_preprocessed)

    pca = PCA(n_components=2)
    reduced = pca.fit_transform(X_preprocessed)

    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(reduced[:, 0], reduced[:, 1], c=labels, cmap='tab10', s=20, alpha=0.6)
    plt.title("DBSCAN Cluster Visualization (PCA-Reduced)")
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.grid(True)
    plt.legend(*scatter.legend_elements(), title="Clusters")
    plt.tight_layout()
    plt.show()

best_dbscan, best_score = evaluate_dbscan(X)
if best_dbscan:
    visualize_dbscan_clusters(best_dbscan, X, preprocessor)
else:
    print("No valid DBSCAN model was found. Please adjust the parameters or check the dataset.")

## Model Interpretation
"""

### Overview
This section interprets the output of the evaluated models — Logistic Regression, Linear Regression, Neural Network, and DBSCAN — providing context for what the results mean and implications for the school district.

---

### Logistic Regression
- **Precision**: 0.6625
- **Recall**: 0.6883
- **F1 Score**: 0.6752

**Interpretation**: Logistic regression performed reasonably well in predicting students who would have a referral the following week. A recall of ~69% means it correctly identified most of the actual positive cases (disruptive students), while a precision of ~66% shows a moderate rate of false positives. Its F1 score (0.6752) balances both measures effectively, making it a good baseline classifier.

---

### Linear Regression
- **RMSE**: 0.7571
- **R²**: -0.2022

**Interpretation**: The model's R² is negative, indicating that it performs worse than simply predicting the mean value for all observations. The linear regression model failed to capture patterns in the data to predict the number of weekly referrals accurately, suggesting the relationship is not linear or lacks strong predictive variables.

---

### Neural Network
- **Precision**: 0.5500
- **Recall**: 0.4286
- **F1 Score**: 0.4818

**Interpretation**: The neural network underperformed compared to logistic regression. Its lower recall (42.86%) indicates a tendency to miss actual referral cases, and its overall F1 score (0.4818) suggests it's less reliable in high-stakes prediction. This may indicate overfitting or sensitivity to data imbalance.

---

### DBSCAN Clustering
- **Best Params**: eps=0.8, min_samples=3
- **Best Silhouette Score**: 0.4634

**Interpretation**: DBSCAN discovered moderately well-separated clusters (silhouette ~0.46). These clusters could correspond to different behavior profiles or referral risk tiers. However, the silhouette score indicates that many points lie near cluster boundaries, suggesting some overlap or noise in student behavior patterns.

---

### General Takeaways
- **Logistic Regression** is the most effective supervised method for this classification task, offering the best balance between false positives and missed referrals.
- **Linear Regression** is ineffective for predicting the number of referrals.
- **Neural Networks** may benefit from more data, better class balancing, or hyperparameter tuning.
- **DBSCAN** shows potential for uncovering behavioral profiles but may require further feature selection or tuning.

**Recommendations**:
- Continue with Logistic Regression for prediction.
- Explore ensemble methods (e.g., Random Forest, Gradient Boosting).
- Investigate feature interactions and apply dimensionality reduction techniques.
- Use DBSCAN output to flag outliers for intervention or manual review.
"""

## 8. Result Cleanup
*(To be completed by team)*